In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/otto-recommender-system/sample_submission.csv
/kaggle/input/otto-recommender-system/test.jsonl
/kaggle/input/otto-recommender-system/train.jsonl
/kaggle/input/otto-full-optimized-memory-footprint/id2type.pkl
/kaggle/input/otto-full-optimized-memory-footprint/train.parquet
/kaggle/input/otto-full-optimized-memory-footprint/type2id.pkl
/kaggle/input/otto-full-optimized-memory-footprint/test.parquet


# 1. Setup Notebook

In [6]:
# import libraries
!pip install polars

import polars as pl
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 72.2 MB/s eta 0:00:0000:0100:01


In [4]:
# Environment variable
INPUT_PATH = Path("../input/otto-full-optimized-memory-footprint")
TRAIN_PATH = INPUT_PATH/"train.parquet"
TEST_PATH = INPUT_PATH/"test.parquet"

# 1. Load Sample Train Data 

In [7]:
# # number of rows in train & test
# with open(TRAIN_PATH, 'r') as f:
#     print(f"Number of records in train dataset: {len(f.readlines())}")
    
# with open(TEST_PATH, 'r') as f:
#     print(f"Number of records in test dataset: {len(f.readlines())}")

In [10]:
# # load sample data
# sample_size = 10000

# sample_train_df = pd.read_json(TRAIN_PATH, nrows= sample_size, lines=True)

# # qc
# print(sample_train_df.shape)
# sample_train_df.head()

(10000, 2)


,session,events
0,0,"[{'aid': 1517085, 'ts': 1659304800025, 'type':..."
1,1,"[{'aid': 424964, 'ts': 1659304800025, 'type': ..."
2,2,"[{'aid': 763743, 'ts': 1659304800038, 'type': ..."
3,3,"[{'aid': 1425967, 'ts': 1659304800095, 'type':..."
4,4,"[{'aid': 613619, 'ts': 1659304800119, 'type': ..."


In [12]:
# # look at first session
# sample_session = sample_train_df.iloc[0]['events']

# print(len(sample_session))
# print(sample_session[:20])

276
[{'aid': 1517085, 'ts': 1659304800025, 'type': 'clicks'}, {'aid': 1563459, 'ts': 1659304904511, 'type': 'clicks'}, {'aid': 1309446, 'ts': 1659367439426, 'type': 'clicks'}, {'aid': 16246, 'ts': 1659367719997, 'type': 'clicks'}, {'aid': 1781822, 'ts': 1659367871344, 'type': 'clicks'}, {'aid': 1152674, 'ts': 1659367885796, 'type': 'clicks'}, {'aid': 1649869, 'ts': 1659369893840, 'type': 'carts'}, {'aid': 461689, 'ts': 1659369898050, 'type': 'carts'}, {'aid': 305831, 'ts': 1659370027105, 'type': 'orders'}, {'aid': 461689, 'ts': 1659370027105, 'type': 'orders'}, {'aid': 362233, 'ts': 1659370064916, 'type': 'clicks'}, {'aid': 1649869, 'ts': 1659370067686, 'type': 'clicks'}, {'aid': 1649869, 'ts': 1659371003682, 'type': 'clicks'}, {'aid': 984597, 'ts': 1659371033243, 'type': 'clicks'}, {'aid': 1649869, 'ts': 1659371042297, 'type': 'clicks'}, {'aid': 803544, 'ts': 1659371044075, 'type': 'clicks'}, {'aid': 1110941, 'ts': 1659371104329, 'type': 'clicks'}, {'aid': 1190046, 'ts': 1659371123063

# 2. Load Sample Test Data

In [13]:
# # load sample data
# sample_size = 10000

# sample_test_df = pd.read_json(TEST_PATH, nrows= sample_size, lines=True)

# # qc
# print(sample_test_df.shape)
# sample_test_df.head()

(10000, 2)


,session,events
0,12899779,"[{'aid': 59625, 'ts': 1661724000278, 'type': '..."
1,12899780,"[{'aid': 1142000, 'ts': 1661724000378, 'type':..."
2,12899781,"[{'aid': 141736, 'ts': 1661724000559, 'type': ..."
3,12899782,"[{'aid': 1669402, 'ts': 1661724000568, 'type':..."
4,12899783,"[{'aid': 255297, 'ts': 1661724000572, 'type': ..."


In [15]:
# # look at first session
# sample_test_session = sample_test_df.iloc[2]['events']

# print(len(sample_test_session))
# print(sample_test_session[:20])

11
[{'aid': 141736, 'ts': 1661724000559, 'type': 'clicks'}, {'aid': 199008, 'ts': 1661724022851, 'type': 'clicks'}, {'aid': 57315, 'ts': 1661724170835, 'type': 'clicks'}, {'aid': 194067, 'ts': 1661724246188, 'type': 'clicks'}, {'aid': 199008, 'ts': 1661780623778, 'type': 'clicks'}, {'aid': 199008, 'ts': 1661781274081, 'type': 'clicks'}, {'aid': 199008, 'ts': 1661781409993, 'type': 'carts'}, {'aid': 199008, 'ts': 1661804151788, 'type': 'clicks'}, {'aid': 199008, 'ts': 1662060028567, 'type': 'clicks'}, {'aid': 199008, 'ts': 1662060064706, 'type': 'clicks'}, {'aid': 918667, 'ts': 1662060160406, 'type': 'clicks'}]


In [18]:
# sample_submission = pd.read_csv(INPUT_PATH/"sample_submission.csv", nrows=1000)

# sample_submission.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


So, for each session, we need to predict the aids for each type(clicks, carts, orders)

Refer this notebook for further EDA:

https://www.kaggle.com/code/edwardcrookenden/otto-getting-started-eda-baseline

# 2. Approaches

1. Co-Visitation Matrix
2. Matrix Factorization - https://www.kaggle.com/code/radek1/matrix-factorization-pytorch-merlin-dataloader
    
2. Time Series
    Visualization - https://www.kaggle.com/code/cdeotte/time-series-eda-users-and-real-sessions

# 3. Load data from Pickle

In [7]:
%%time
train = pl.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
test = pl.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

# qc
print(train.shape)
print(test.shape)

train.head()

(216716096, 4)
(6928123, 4)
CPU times: user 7.41 s, sys: 2.92 s, total: 10.3 s
Wall time: 26.7 s


session,aid,ts,type
i32,i32,i32,u8
0,1517085,1659304800,0
0,1563459,1659304904,0
0,1309446,1659367439,0
0,16246,1659367719,0
0,1781822,1659367871,0


In [8]:
%%time
consecutive_aid_data = (pl.concat([train, test]).groupby('session')\
                       .agg([pl.col('aid'), pl.col('aid').shift(-1).alias('aid_next')])\
                       .explode(['aid', 'aid_next']).drop_nulls())[['aid', 'aid_next']]

print(consecutive_aid_data.shape)

(209072637, 2)
CPU times: user 50.4 s, sys: 15 s, total: 1min 5s
Wall time: 27.5 s


In [9]:
train.head()

session,aid,ts,type
i32,i32,i32,u8
0,1517085,1659304800,0
0,1563459,1659304904,0
0,1309446,1659367439,0
0,16246,1659367719,0
0,1781822,1659367871,0


In [11]:
train['aid'].n_unique()

1855603

# Rece